# Importing the libs

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the datasets

In [2]:
# We won't be using this dataset.
movies = pd.read_csv('data/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('data/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('data/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Prepare the training & test sets

In [3]:
training_set = pd.read_csv('data/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('data/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

# Get the number of users and movies

In [4]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_users

943

In [5]:
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))
nb_movies

1682

# Converting & preparing data

In [6]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
      
    # Get the current user's movie ids
    id_movies = data[:, 1][data[:, 0] == id_users]
    
    # Get the current user's rating scores 
    id_ratings = data[:, 2][data[:, 0] == id_users]
    
    ratings = np.zeros(nb_movies)
    # Assign the ratings to movies, which they belong
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Convert data to tensors

In [7]:
training_set = torch.FloatTensor(training_set)
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [8]:
test_set = torch.FloatTensor(test_set)
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

# Create the SAE Class

In [9]:
# We need to create a class to specify and define the behaviours of our AE (to give instructions)
# Object of this class will be an AE
# Inheritance to use all the functions of the parent class
# SAE -> Stacked Auto Encoders
# We have several hidden layers -> several encodings of the input vector features
class SAE(nn.Module):
    def __init__(self, ): # Put nothing after the comma, because we have inheritance
        super(SAE, self).__init__() # Get the functions/methods of the module
        ## nn is module, Linear and Sigmoid are classes of the nn module
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        # The next function get the output of the last function as input
        # nb_movies - 20neuron - 10neuron - 20 neuron - nb_movies -> these are the layers
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [10]:
# Create instances and define parameters
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training

In [11]:
# Define number of epoch
nb_epoch = 200

In [12]:
for epoch in range (1, nb_epoch + 1):
    train_loss = 0
    
    #we will use RMSE at the end and RMSE is a float. That's why we use float 
    s = 0. # number of users who rated at least one movie
    
    for id_user in range (nb_users):
        
        # Add a fake dimension for functions
        # index of fake dimension is 0
        # we create batch
        input = Variable(training_set[id_user]).unsqueeze(0)
        
        target = input.clone()
        if torch.sum(target.data>0) > 0:
            output = sae(input)
            # GD won't be computed in target, it'll computed just in input vector
            target.require_grad = False
            
            # zero values don't have impact on the update of the different weights
            # before the loss we make them 0 to save comp power
            output[target == 0] = 0 
            
            # calculate the loss func.
            loss = criterion(output, target)
            
            # number of all movies divided by number of rated movies
            mean_corrector = nb_movies/float(torch.sum(target.data>0) + 1e-10)
            
            # increase or decrease the weights? ->backward method
            # direction of the optimization
            loss.backward()
            
            # loss data is the square error, that's why we take sqrt to get one degree loss
            train_loss  += np.sqrt(loss.data*mean_corrector)
            
            # if user rated, increase s
            s += 1.
            
            
            optimizer.step() # intensity/amount of the updates
    print('epoch :' + str(epoch) + ' loss: ' + str(train_loss/s))

epoch :1 loss: tensor(1.7713)
epoch :2 loss: tensor(1.0968)
epoch :3 loss: tensor(1.0533)
epoch :4 loss: tensor(1.0381)
epoch :5 loss: tensor(1.0310)
epoch :6 loss: tensor(1.0264)
epoch :7 loss: tensor(1.0240)
epoch :8 loss: tensor(1.0219)
epoch :9 loss: tensor(1.0209)
epoch :10 loss: tensor(1.0193)
epoch :11 loss: tensor(1.0191)
epoch :12 loss: tensor(1.0185)
epoch :13 loss: tensor(1.0181)
epoch :14 loss: tensor(1.0176)
epoch :15 loss: tensor(1.0174)
epoch :16 loss: tensor(1.0169)
epoch :17 loss: tensor(1.0168)
epoch :18 loss: tensor(1.0164)
epoch :19 loss: tensor(1.0165)
epoch :20 loss: tensor(1.0160)
epoch :21 loss: tensor(1.0165)
epoch :22 loss: tensor(1.0157)
epoch :23 loss: tensor(1.0160)
epoch :24 loss: tensor(1.0157)
epoch :25 loss: tensor(1.0156)
epoch :26 loss: tensor(1.0156)
epoch :27 loss: tensor(1.0153)
epoch :28 loss: tensor(1.0153)
epoch :29 loss: tensor(1.0137)
epoch :30 loss: tensor(1.0119)
epoch :31 loss: tensor(1.0096)
epoch :32 loss: tensor(1.0098)
epoch :33 loss: t

# Testing the SAE

In [13]:
test_loss = 0
#we will use RMSE at the end and RMSE is a float. That's why we use float 
s = 0. # number of users who rated at least one movie
for id_user in range(nb_users):
    # Add a fake dimension for functions
    # index of fake dimension is 0
    # we create batch
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0) 
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    # GD won't be computed in target, it'll computed just in input vector
    target.require_grad = False
    # zero values don't have impact on the update of the different weights
    # before the loss we make them 0 to save comp power
    output[target == 0] = 0
    # calculate the loss func.
    loss = criterion(output, target)
    # number of all movies divided by number of rated movies
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    # loss data is the square error, that's why we take sqrt to get one degree loss
    test_loss += np.sqrt(loss.data*mean_corrector)
    # if user rated, increase s
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9585)
